In [1]:
'''
This is an example script to solve a Laplace problem using OpenCMISS-Iron calls in python
ike all python examples with importing modules. In addition to sys, os and the CMISS module, 
math is imported as we will be using the math.sqrt function later on.
In mathematics and physics, Laplace's equation is a second-order partial differential equation 
named after Pierre-Simon Laplace who first studied its properties.
'''
import sys, os
# Intialise OpenCMISS-Iron
from opencmiss.iron import iron
#Note that importing iron from opencmiss initialises library with default values and
#seeds random generator with a new value

In [2]:
# Set problem parameters
height = 1.0
width = 2.0
length = 3.0

#We need constant identificators for various 
#library functional points
(coordinateSystemUserNumber,
    regionUserNumber,
    basisUserNumber,
    generatedMeshUserNumber,
    meshUserNumber,
    decompositionUserNumber,
    geometricFieldUserNumber,
    equationsSetFieldUserNumber,
    dependentFieldUserNumber,
    equationsSetUserNumber,
    problemUserNumber) = range(1,12)

In [3]:
#initialise parameters of the model
numberGlobalXElements = 5
numberGlobalYElements = 5
numberGlobalZElements = 5


In [4]:
# Get the computational nodes information
# Note that in order for the model to utillise
# parallel computations the code must be running
# under the supervision of an MPI library
# and as such must be executed with mpirun/mpiexec assistance
# If mpirun/mpiexec is not used, number of nodes witll be 1 and
# only the master node will be created
numberOfComputationalNodes = iron.ComputationalNumberOfNodesGet()
computationalNodeNumber = iron.ComputationalNodeNumberGet()

In [5]:

# The first step in our example will be to initialise OpenCMISS an to set up a region 
# and coordinate system for our 3D mesh.

# Creation a RC coordinate system
coordinateSystem = iron.CoordinateSystem()
coordinateSystem.CreateStart(coordinateSystemUserNumber)
coordinateSystem.dimension = 3
coordinateSystem.CreateFinish()

# Create a region
region = iron.Region()
region.CreateStart(regionUserNumber,iron.WorldRegion)
region.label = "LaplaceRegion"
region.coordinateSystem = coordinateSystem
region.CreateFinish()

In [6]:
# In this example we will use the generated mesh capabilities of OpenCMISS to 
# generate our 3D mesh. The first thing we need to do is create a basis function 
# for the mesh. We will define a trilinear Lagrange basis.
# Create a tri-linear lagrange basis
basis = iron.Basis()
basis.CreateStart(basisUserNumber)
basis.type = iron.BasisTypes.LAGRANGE_HERMITE_TP
basis.numberOfXi = 3
basis.interpolationXi = [iron.BasisInterpolationSpecifications.LINEAR_LAGRANGE]*3
basis.quadratureNumberOfGaussXi = [2]*3
basis.CreateFinish()

In [7]:
# We can now create a generated mesh. We will create a regular mesh of size 
# width x height x length and divide the mesh into numberOfXElements in the X direction,
# numberOfYElements in the Y direction and numberOfZElements in the Z direction. 
# Note that when we finish generating the mesh we have a mesh object returned to us. 
# This mesh object is just the same as if we had manually created the regular mesh.

#  Create a generated mesh
generatedMesh = iron.GeneratedMesh()
generatedMesh.CreateStart(generatedMeshUserNumber,region)
generatedMesh.type = iron.GeneratedMeshTypes.REGULAR
generatedMesh.basis = [basis]
generatedMesh.extent = [width,height,length]
generatedMesh.numberOfElements = [numberGlobalXElements,numberGlobalYElements,numberGlobalZElements]

In [8]:
# Let's double check the number of computational nodes we use
print(numberOfComputationalNodes)

1


In [9]:
mesh = iron.Mesh()
generatedMesh.CreateFinish(meshUserNumber,mesh)


In [10]:
# Once the mesh has been created we can decompose it into a number of domains in order to allow for 
# parallelism. We choose the options to let OpenCMISS calculate the best way to break up the mesh. 
# We also set the number of domains to be equal to the number of computational nodes this example is running on.
# Note that if MPI infrastructure is not used, only single domain will ve created

# Create a decomposition for the mesh
decomposition = iron.Decomposition()
decomposition.CreateStart(decompositionUserNumber,mesh)
decomposition.type = iron.DecompositionTypes.CALCULATED
decomposition.numberOfDomains = numberOfComputationalNodes
decomposition.CreateFinish()

In [11]:
# Now that the mesh has been decomposed we are in a position to create fields. The first field we need to 
# create is the geometry field. Here we create a field and set the field’s mesh decomposition to the 
# decomposed mesh that we have just created. We can choose exact how each component of the field is 
# interpolated by setting component mesh component to be the mesh components that we created for the mesh. 
# For this example we only have one mesh component. Once we have finished creating the field we can change 
# the field DOFs to give us our geometry. Since this mesh has been generated we can use the generated mesh 
# object to calculate the geometric parameters of the regular mesh.

# Create a field for the geometry
geometricField = iron.Field()
geometricField.CreateStart(geometricFieldUserNumber,region)
geometricField.meshDecomposition = decomposition
geometricField.ComponentMeshComponentSet(iron.FieldVariableTypes.U,1,1)
geometricField.ComponentMeshComponentSet(iron.FieldVariableTypes.U,2,1)
geometricField.ComponentMeshComponentSet(iron.FieldVariableTypes.U,3,1)
geometricField.CreateFinish()

In [12]:
# Set geometry from the generated mesh
generatedMesh.GeometricParametersCalculate(geometricField)

In [13]:
# We are now in a position to define the type of physics that we wish to solve. This is done by creating an 
# equations set which is a contianer object for all the parameters we need to describe the physics. 
# Here we create a standard Laplace equations set.

# {\displaystyle \nabla ^{2}f={\frac {\partial ^{2}f}{\partial x^{2}}}+{\frac {\partial ^{2}f}{\partial y^{2}}}+{\frac {\partial ^{2}f}{\partial z^{2}}}=0.}

# Create standard Laplace equations set
equationsSetField = iron.Field()
equationsSet = iron.EquationsSet()
equationsSetSpecification = [iron.EquationsSetClasses.CLASSICAL_FIELD,
        iron.EquationsSetTypes.LAPLACE_EQUATION,
        iron.EquationsSetSubtypes.STANDARD_LAPLACE]
equationsSet.CreateStart(equationsSetUserNumber,region,geometricField,
        equationsSetSpecification,equationsSetFieldUserNumber,equationsSetField)
equationsSet.CreateFinish()

In [14]:
# For the Laplace equation we need a dependent field (our solution).
# Here we do not define a field before the create starts and so we let OpenCMISS create an appropriate 
# dependent field for the Laplace equations being described. Once the fields have been created we can
# set the field DOF values. 

# Create dependent field
dependentField = iron.Field()
equationsSet.DependentCreateStart(dependentFieldUserNumber,dependentField)
dependentField.DOFOrderTypeSet(iron.FieldVariableTypes.U,iron.FieldDOFOrderTypes.SEPARATED)
dependentField.DOFOrderTypeSet(iron.FieldVariableTypes.DELUDELN,iron.FieldDOFOrderTypes.SEPARATED)
equationsSet.DependentCreateFinish()

# Initialise dependent field
dependentField.ComponentValuesInitialiseDP(iron.FieldVariableTypes.U,iron.FieldParameterSetTypes.VALUES,1,0.5)


In [15]:
# Create equations 
equations = iron.Equations()
equationsSet.EquationsCreateStart(equations)
equations.sparsityType = iron.EquationsSparsityTypes.SPARSE
equations.outputType = iron.EquationsOutputTypes.NONE
equationsSet.EquationsCreateFinish()


In [16]:
# Now we are ready to set up a problem to be solved by OpenCMISS
# Create Laplace problem

problem = iron.Problem()
problemSpecification = [iron.ProblemClasses.CLASSICAL_FIELD,
        iron.ProblemTypes.LAPLACE_EQUATION,
        iron.ProblemSubtypes.STANDARD_LAPLACE]
problem.CreateStart(problemUserNumber, problemSpecification)
problem.CreateFinish()

In [17]:
# OpenCMISS control loop is a "supervisor" for the computational process.
# Create control loops
problem.ControlLoopCreateStart()
problem.ControlLoopCreateFinish()

In [18]:
# Now we are ready to setup a solver for the problem. We need to specify solver type (iterative) and tolerances 
# in order for the computational loop to maintain control over the solution

# Create problem solver
solver = iron.Solver()
problem.SolversCreateStart()
problem.SolverGet([iron.ControlLoopIdentifiers.NODE],1,solver)
solver.outputType = iron.SolverOutputTypes.SOLVER
solver.linearType = iron.LinearSolverTypes.ITERATIVE
solver.linearIterativeAbsoluteTolerance = 1.0E-12
solver.linearIterativeRelativeTolerance = 1.0E-12
problem.SolversCreateFinish()


In [19]:
# Create solver equations and add equations set to solver equations
solver = iron.Solver()
solverEquations = iron.SolverEquations()
problem.SolverEquationsCreateStart()
problem.SolverGet([iron.ControlLoopIdentifiers.NODE],1,solver)
solver.SolverEquationsGet(solverEquations)
solverEquations.sparsityType = iron.SolverEquationsSparsityTypes.SPARSE
equationsSetIndex = solverEquations.EquationsSetAdd(equationsSet)
problem.SolverEquationsCreateFinish()

In [20]:
# The Dirichlet problem for Laplace's equation consists of finding a solution φ on some domain D such that φ on 
# the boundary of D is equal to some given function. Since the Laplace operator appears in the heat equation, 
# one physical interpretation of this problem is as follows: fix the temperature on the boundary of the domain 
# according to the given specification of the boundary condition. Allow heat to flow until a stationary state is 
# reached in which the temperature at each point on the domain doesn't change anymore. 
# The temperature distribution in the interior will then be given by the solution to the corresponding 
# Dirichlet problem.

# Create boundary conditions and set first and last nodes to 0.0 and 1.0
boundaryConditions = iron.BoundaryConditions()
solverEquations.BoundaryConditionsCreateStart(boundaryConditions)
firstNodeNumber=1
nodes = iron.Nodes()
region.NodesGet(nodes)
lastNodeNumber = nodes.numberOfNodes
firstNodeDomain = decomposition.NodeDomainGet(firstNodeNumber,1)
lastNodeDomain = decomposition.NodeDomainGet(lastNodeNumber,1)
if firstNodeDomain == computationalNodeNumber:
    boundaryConditions.SetNode(dependentField,iron.FieldVariableTypes.U,1,1,firstNodeNumber,1,iron.BoundaryConditionsTypes.FIXED,0.0)
if lastNodeDomain == computationalNodeNumber:
    boundaryConditions.SetNode(dependentField,iron.FieldVariableTypes.U,1,1,lastNodeNumber,1,iron.BoundaryConditionsTypes.FIXED,1.0)
solverEquations.BoundaryConditionsCreateFinish()

In [21]:
# Solve the problem
problem.Solve()

In [22]:
'''
And the results of the run are:
Solver:
  Solver index = 1

Solver pre-solve:

Solver equations solve:
Solver matrices assembly                          0.000000E+00  0.000000E+00  0.000000E+00
Solver RHS assembly                               0.000000E+00  0.000000E+00  0.000000E+00

Linear iterative solver parameters:
Final number of iterations = 65
Final residual norm =    9.5010877749490898E-013
Converged Reason = PETSc converged ATol

Solver solution vectors:
Number of solution vectors = 1
Solution vector for solver matrix : 1
Vector(:)          :  0.454768E+00  0.453061E+00  0.474333E+00  0.479934E+00  0.482290E+00  0.316870E+00  0.409378E+00  0.462585E+00
                      0.472722E+00  0.480318E+00  0.482162E+00  0.402008E+00  0.428823E+00  0.460520E+00  0.473907E+00  0.480185E+00
                      0.482271E+00  0.433056E+00  0.443886E+00  0.462155E+00  0.474202E+00  0.480324E+00  0.482290E+00  0.445737E+00
                      0.451788E+00  0.464094E+00  0.474444E+00  0.480395E+00  0.482433E+00  0.449290E+00  0.454204E+00  0.464855E+00
                      0.474531E+00  0.480518E+00  0.482091E+00  0.515997E+00  0.439936E+00  0.470055E+00  0.478714E+00  0.484677E+00
                      0.486369E+00  0.450287E+00  0.454469E+00  0.467957E+00  0.479295E+00  0.484600E+00  0.486426E+00  0.450036E+00
                      0.457122E+00  0.469658E+00  0.479334E+00  0.484718E+00  0.486441E+00  0.455945E+00  0.460982E+00  0.471032E+00
                      0.479634E+00  0.484782E+00  0.486447E+00  0.460150E+00  0.463890E+00  0.472092E+00  0.479883E+00  0.484851E+00
                      0.486268E+00  0.461582E+00  0.464923E+00  0.472480E+00  0.480031E+00  0.484658E+00  0.487089E+00  0.471228E+00
                      0.488941E+00  0.487827E+00  0.493293E+00  0.496779E+00  0.498081E+00  0.486044E+00  0.485514E+00  0.488588E+00
                      0.493186E+00  0.496810E+00  0.498062E+00  0.484843E+00  0.485417E+00  0.488683E+00  0.493308E+00  0.496818E+00
                      0.498090E+00  0.483959E+00  0.485250E+00  0.488910E+00  0.493397E+00  0.496893E+00  0.498314E+00  0.483784E+00
                      0.485276E+00  0.489069E+00  0.493481E+00  0.496932E+00  0.498933E+00  0.483796E+00  0.485308E+00  0.489169E+00
                      0.493323E+00  0.497789E+00  0.495580E+00  0.504420E+00  0.502211E+00  0.506677E+00  0.510831E+00  0.514692E+00
                      0.516204E+00  0.501067E+00  0.503068E+00  0.506519E+00  0.510931E+00  0.514724E+00  0.516216E+00  0.501686E+00
                      0.503107E+00  0.506603E+00  0.511090E+00  0.514750E+00  0.516041E+00  0.501910E+00  0.503182E+00  0.506692E+00
                      0.511317E+00  0.514583E+00  0.515157E+00  0.501938E+00  0.503190E+00  0.506814E+00  0.511412E+00  0.514486E+00
                      0.513956E+00  0.501919E+00  0.503221E+00  0.506707E+00  0.512173E+00  0.511059E+00  0.528772E+00  0.512911E+00
                      0.515342E+00  0.519969E+00  0.527520E+00  0.535077E+00  0.538418E+00  0.513732E+00  0.515149E+00  0.520117E+00
                      0.527908E+00  0.536110E+00  0.539850E+00  0.513553E+00  0.515218E+00  0.520366E+00  0.528968E+00  0.539018E+00
                      0.544055E+00  0.513559E+00  0.515282E+00  0.520666E+00  0.530342E+00  0.542878E+00  0.549964E+00  0.513574E+00
                      0.515400E+00  0.520705E+00  0.532043E+00  0.545531E+00  0.549713E+00  0.513631E+00  0.515323E+00  0.521286E+00
                      0.529945E+00  0.560064E+00  0.484003E+00  0.517909E+00  0.519482E+00  0.525469E+00  0.535145E+00  0.545796E+00
                      0.550710E+00  0.517567E+00  0.519605E+00  0.525556E+00  0.535906E+00  0.548212E+00  0.554263E+00  0.517710E+00
                      0.519676E+00  0.525798E+00  0.537845E+00  0.556114E+00  0.566944E+00  0.517729E+00  0.519815E+00  0.526093E+00
                      0.539480E+00  0.571177E+00  0.597992E+00  0.517838E+00  0.519682E+00  0.527278E+00  0.537415E+00  0.590622E+00
                      0.683130E+00  0.517710E+00  0.520066E+00  0.525667E+00  0.546939E+00  0.545232E+00
Total time for solve                              0.000000E+00  0.000000E+00  0.000000E+00

Solver post-solve:
'''


'\nAnd the results of the run are:\nSolver:\n  Solver index = 1\n\nSolver pre-solve:\n\nSolver equations solve:\nSolver matrices assembly                          0.000000E+00  0.000000E+00  0.000000E+00\nSolver RHS assembly                               0.000000E+00  0.000000E+00  0.000000E+00\n\nLinear iterative solver parameters:\nFinal number of iterations = 65\nFinal residual norm =    9.5010877749490898E-013\nConverged Reason = PETSc converged ATol\n\nSolver solution vectors:\nNumber of solution vectors = 1\nSolution vector for solver matrix : 1\nVector(:)          :  0.454768E+00  0.453061E+00  0.474333E+00  0.479934E+00  0.482290E+00  0.316870E+00  0.409378E+00  0.462585E+00\n                      0.472722E+00  0.480318E+00  0.482162E+00  0.402008E+00  0.428823E+00  0.460520E+00  0.473907E+00  0.480185E+00\n                      0.482271E+00  0.433056E+00  0.443886E+00  0.462155E+00  0.474202E+00  0.480324E+00  0.482290E+00  0.445737E+00\n                      0.451788E+00  0.4

In [23]:
# Now we want to have the results of the run be stored for analysis and later use

# Export results
baseName = "laplace"
dataFormat = "PLAIN_TEXT"
fml = iron.FieldMLIO()
fml.OutputCreate(mesh, "", baseName, dataFormat)
fml.OutputAddFieldNoType(baseName+".geometric", dataFormat, geometricField,
    iron.FieldVariableTypes.U, iron.FieldParameterSetTypes.VALUES)
fml.OutputAddFieldNoType(baseName+".phi", dataFormat, dependentField,
    iron.FieldVariableTypes.U, iron.FieldParameterSetTypes.VALUES)
fml.OutputWrite("LaplaceExample.xml")
fml.Finalise()

In [24]:
# Let the library know that you are done with computations and the resources allocated for the problem
# can now be released
iron.Finalise()